In [132]:
import time
import requests
from time import sleep
import re, requests, csv
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
keys = Keys()

In [133]:
from selenium.webdriver.common.by import By

In [134]:
next_btn = []
for i in range(2, 12):
    next_btn.append(f'a:nth-child({i})')
next_btn.append("a.fAUKm1ewwo._2Ar8-aEUTq")

In [213]:
def get_reviews(address, stop):
    driver = webdriver.Chrome()
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--hide-scrollbars")
    driver.get(address)
    page = requests.get(address)

    # stop이 총 리뷰 수보다 많다면, 총 리뷰 수로 계산하기

    print(f"{address} 페이지 리뷰 크롤링을 시작합니다.")

    count = 0

    review_list = []

    driver.implicitly_wait(3)
    
    # 리뷰 보기 버튼 클릭
    el = driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a')
    actions = webdriver.ActionChains(driver).move_to_element(el).click().perform()
    print("리뷰 창으로 넘어감")
    time.sleep(10)

    # 리뷰 낮은 순 보기 버튼 클릭
    el = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[4]/a')
    actions = webdriver.ActionChains(driver).move_to_element(el).perform()
    actions2 = webdriver.ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
    el.send_keys(Keys.ENTER)
    print("리뷰 낮은 순으로 정렬")
    time.sleep(3)

    while (len(review_list) < stop):
        if (count % 20 == 0):
            el = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > " + str(next_btn[(count // 20) % 11]) + '') # 리뷰 인덱스 페이지 이동
            actions = webdriver.ActionChains(driver).move_to_element(el)
            driver.implicitly_wait(3)
            actions.perform()
            el.click()
            time.sleep(2)
            html = driver.page_source
            soup = bs(html, "html.parser")
            reviews = soup.find_all('div', class_='YEtwtZFLDz')
            print(f"불러온 리뷰 : {len(reviews)}개")
        review = reviews[count % 20].text
        review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        if ("더보기" in review):
            review = soup.find_all('div', class_="YEtwtZFLDz.vlXMQEAtPR")
            review = review[count % 20].text
            review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        review_list.append(review)
        count = count + 1
        print(f'{count}번째 리뷰 가져옴')
    driver.quit()
    return review_list

In [197]:
PAGES = ['https://brand.naver.com/skinfood/products/7926935977',
        'https://smartstore.naver.com/kikoluggage/products/6399548845?n_media=11068&n_query=%EA%B8%B0%EB%82%B4%EC%9A%A9%EC%BA%90%EB%A6%AC%EC%96%B4&n_rank=1&n_ad_group=grp-a001-02-000000033272787&n_ad=nad-a001-02-000000235065323&n_campaign_type=2&n_mall_id=shc2016&n_mall_pid=6399548845&n_ad_group_type=2&n_match=3'
         ]

In [214]:
review_pages = {}
for i in PAGES:
    review_pages[i] = get_reviews(i, 30)

https://brand.naver.com/skinfood/products/7926935977 페이지 리뷰 크롤링을 시작합니다.
리뷰 창으로 넘어감
리뷰 낮은 순으로 정렬
불러온 리뷰 : 20개
1번째 리뷰 가져옴
2번째 리뷰 가져옴
3번째 리뷰 가져옴
4번째 리뷰 가져옴
5번째 리뷰 가져옴
6번째 리뷰 가져옴
7번째 리뷰 가져옴
8번째 리뷰 가져옴
9번째 리뷰 가져옴
10번째 리뷰 가져옴
11번째 리뷰 가져옴
12번째 리뷰 가져옴
13번째 리뷰 가져옴
14번째 리뷰 가져옴
15번째 리뷰 가져옴
16번째 리뷰 가져옴
17번째 리뷰 가져옴
18번째 리뷰 가져옴
19번째 리뷰 가져옴
20번째 리뷰 가져옴
불러온 리뷰 : 20개
21번째 리뷰 가져옴
22번째 리뷰 가져옴
23번째 리뷰 가져옴
24번째 리뷰 가져옴
25번째 리뷰 가져옴
26번째 리뷰 가져옴
27번째 리뷰 가져옴
28번째 리뷰 가져옴
29번째 리뷰 가져옴
30번째 리뷰 가져옴
https://smartstore.naver.com/kikoluggage/products/6399548845?n_media=11068&n_query=%EA%B8%B0%EB%82%B4%EC%9A%A9%EC%BA%90%EB%A6%AC%EC%96%B4&n_rank=1&n_ad_group=grp-a001-02-000000033272787&n_ad=nad-a001-02-000000235065323&n_campaign_type=2&n_mall_id=shc2016&n_mall_pid=6399548845&n_ad_group_type=2&n_match=3 페이지 리뷰 크롤링을 시작합니다.
리뷰 창으로 넘어감
리뷰 낮은 순으로 정렬
불러온 리뷰 : 20개
1번째 리뷰 가져옴
2번째 리뷰 가져옴
3번째 리뷰 가져옴
4번째 리뷰 가져옴
5번째 리뷰 가져옴
6번째 리뷰 가져옴
7번째 리뷰 가져옴
8번째 리뷰 가져옴
9번째 리뷰 가져옴
10번째 리뷰 가져옴
11번째 리뷰 가져옴
12번째 리뷰 가져옴
13번째 리뷰 가져옴
14번째 리뷰 가져옴
15번째 리뷰

In [218]:
for i in review_pages[PAGES[1]]:
    print(i)

한달사용기AS 수리도 1년 보장에 홈페이지에 캐리어 튼튼하다고 광고하길래 키코 회사 믿고 캐리어 주문했습니다 꼼꼼히 검수해서 보내주시겠거니 생각하고 그대로 보관하다가 여행 하루 전날 포장 뜯어 캐리어 확인해보니 이 상태네요 누가봐도 제작 과정에서 열처리 잘못 들어가서 한쪽이 찌그러진 상태인데 너무 놀라 문의 남기니 고객이 사용하다 그런거 아니냐 택배사에서 옮기다 찌그러진거다 변명하시더라구요 자기들은 검수 꼼꼼히 한다면서고객이 자기가 돈주고 산 캐리어를 저렇게 만들려면 어디 옥상에서 일부러 떨어트렸을까요 아니면 조금만 부딪혀도 찌그러지는 캐리어 수준이었을까요 종이 택배 박스가 온전한 상태로 어디 하나 찌그러진 곳이 없는데 내부에 담겨 있던 캐리어는 택배 기사님이 배송하시다가 찌그러트리셨나봐요 검수를 완벽하게 해서 배송했는데 이런 제품이 왔군요 진상고객 프레임 씌우고 택배기사탓 하기 전에 검수 먼저 꼼꼼하게 해서 보내주시길 바라요ㅎㅎ 리뷰에 저랑 똑같이 나중에 열어봤다가 불량 상품 그대로 쓰시는 분 계시던데 그거 보고 구매하지 말걸 후회하면서 다른 분들은 미리 도망가시라고 리뷰 남깁니다 추가로 구매한 보조배터리 충전도 안 되고 뭐 하나 제대로 된 제품들이 없네요다른 분들은 정말 고민해서 구매하세요마지막 사진은 제가 정말 한번도 사용하지 않은 캐리어라는거 인증 위해 올립니다
생각보가 무겁고 차콜은 왜 광이 벗겨져있나요 티안나게 살짝이 아니라 크게 크게 벗겨져 있네요 급하게 시킨거라 어쩔수 없이 사용하는데 제가 제대로 인지를 못해서 그런건지 지퍼없는게 물건 넣고 빼고 너무 불편하네요
상태 최악입니다송장오류부터 시작해서 상품 상태까지 하자가 많아 너무 실망이였습니다 다시는 구매할 일이 없을 것 같네요 바로 다음 날 사용해야하기에 어쩔 수 없이 구매확정했습니다모서리 부분 총 3곳에 기포자국 같은 이상한 무늬들이 있고 전면의 오른쪽 밑 모서리는 찌그러졌고 닫아도 벌려져 있는 캐리어가 왔습니다 제발 출고 전에 상품 검수 좀 제대로 하시고 보내시기 바랍니다
상품을 받았는데바퀴